# add header

In [1]:
import pandas as pd

# Read the .out file
df = pd.read_csv("20170101.csv.out", header=None, delimiter=",")  # header=None treats first row as data

# Define the column names
columns = ["vehicle_id", "gpsvalid", "lat", "lon", "timestamp", "speed", "heading", "for_hire_light", "engine_acc"]

# Assign the column names to the dataframe
df.columns = columns

# Save as a proper CSV with header
df.to_csv("taxi.csv", index=False)

print("File converted to taxi.csv with header")


File converted to taxi.csv with header


# cleaning part

In [ ]:
import pandas as pd

df = pd.read_csv("taxi.csv")

# Step 1: Group and filter like you already did
dup_all_zero = (
    df.groupby("vehicle_id")
      .agg(
          count_rows=("speed", "size"),
          all_zero=("speed", lambda x: (x == 0).all())
      )
)

dup_all_zero = dup_all_zero[(dup_all_zero["all_zero"]) & (dup_all_zero["count_rows"] > 1)]

# Step 2: Save to all_speed_is_0.csv

# Step 3: Drop rows in df where vehicle_id is in broke_car list
df = df[~df["vehicle_id"].isin(dup_all_zero.index)]

# Step 4: Overwrite original file (permanent)
df.to_csv("vehicle_data_clean.csv", index=False)

In [12]:
import pandas as pd

# Step 1: Load your CSV
df = pd.read_csv("vehicle_data_clean.csv")

# Step 2: Identify vehicles that never got a customer
# for_hire_light = 1 means the taxi had no passenger (for hire), 
# so we check if it ever switched to 1 -> 0 (got a customer)
vehicles_never_hired = df.groupby("vehicle_id")["for_hire_light"].max()
vehicles_never_hired = vehicles_never_hired[vehicles_never_hired == 0].index.tolist()

print("Vehicles that never got any customer:", len(vehicles_never_hired))
# Step 3: Remove these vehicles from the dataframe
df_filtered = df[~df["vehicle_id"].isin(vehicles_never_hired)]
print("how manu row left",len(df_filtered))
# Step 4: Save the filtered dataframe to a new CSV
df_filtered.to_csv("vehicle_data_clean.csv", index=False)

print(f"Filtered CSV saved as 'vehicle_data_filtered.csv' with {len(df_filtered)} rows.")


Vehicles that never got any customer: 431
how manu row left 2030108
Filtered CSV saved as 'vehicle_data_filtered.csv' with 2030108 rows.


In [13]:
import pandas as pd

# Load dataset
df = pd.read_csv("vehicle_data_clean.csv")

# Step 1: Check NaN counts
print("NaN counts per column before dropping:")
print(df.isna().sum())
print("\nTotal NaN values before dropping:", df.isna().sum().sum())

# Step 2: Drop all rows that contain any NaN
df_clean = df.dropna()

# Step 3: Check again after cleaning
print("\nNaN counts per column after dropping:")
print(df_clean.isna().sum())
print("\nTotal NaN values after dropping:", df_clean.isna().sum().sum())

# (Optional) Save cleaned dataset
df_clean.to_csv("vehicle_data_clean.csv", index=False)

NaN counts per column before dropping:
vehicle_id        0
gpsvalid          0
lat               0
lon               0
timestamp         0
speed             0
heading           0
for_hire_light    0
engine_acc        0
dtype: int64

Total NaN values before dropping: 0

NaN counts per column after dropping:
vehicle_id        0
gpsvalid          0
lat               0
lon               0
timestamp         0
speed             0
heading           0
for_hire_light    0
engine_acc        0
dtype: int64

Total NaN values after dropping: 0


In [15]:
# Remove rows where speed > 300
df = df[df["speed"] <= 300].reset_index(drop=True)

# Save cleaned dataset to a new CSV
df.to_csv("vehicle_data_clean.csv", index=False)

# feature engineer part build training set